# Make aliquot_to_patient_ID mapping file (Proteomics)

In [3]:
import pandas as pd
import numpy as np

# Get mapper for ids

Create a df that maps aliquot Ids (Ex: CPT0001580009) to sample IDs (Ex: C3L-00001-02) which are known to be tumor or normal samples. Then sample IDs are mapped to case IDs (Ex: C3L-00001) which we call Patient_ID.

In [7]:
mapper_1 = "pdc_aliquot_2021-03-02_15_58.tsv.xlsx" 
m_path = "../../../input/"+mapper_1

In [8]:
map_df = pd.read_excel(m_path, na_values = 'NA')
map_df

,type,project_id,submitter_id,samples.submitter_id#1,samples.submitter_id#2,samples.submitter_id#3,samples.submitter_id#4,samples.submitter_id#5,analyte_type
0,aliquot,CPTAC-3,CPT0053040004,9f905736-f662-41d6-b3ac-16758d,,,,,protein
1,aliquot,CPTAC-3,CPT0052940004,93e30fd5-e57e-4503-a175-863c7d,,,,,protein
2,aliquot,CPTAC-3,CPT0052170004,5a84eae1-197e-4463-ad65-59becc,,,,,protein
3,aliquot,CPTAC-3,CPT0051690004,2f2e5477-42a4-4906-a943-bf7f80,,,,,protein
4,aliquot,CPTAC-3,CPT0001580009,C3L-00001-02,,,,,protein
...,...,...,...,...,...,...,...,...,...
3070,aliquot,CPTAC-3,CPT032238 0003,C3N-05923-01,,,,,protein
3071,aliquot,CPTAC-3,CPT032239 0003,C3N-05923-09,,,,,protein
3072,aliquot,CPTAC-3,CPT032253 0003,C3N-05929-03,,,,,protein
3073,aliquot,CPTAC-3,CPT032254 0003,C3N-05929-05,,,,,protein


In [9]:
# remove spaces (aliquot IDs with a space or without a space are the same -Mathangi email)
map_df['submitter_id'] = map_df['submitter_id'].replace(" ", "", regex = True)

In [10]:
# Use samples.submitter_id#1 because it always has a value and represents the type of sample for any run in the aliquot 
# (each row is either tumor or normal in pdc_aliquot_2021-03-02_15_58.tsv) 

map_df = map_df.rename(columns = {'submitter_id':'aliquot_ID', 'samples.submitter_id#1':'sample_ID',
                                      'samples.submitter_id#2':'2', 'samples.submitter_id#3':'3',
                                       'samples.submitter_id#4':'4', 'samples.submitter_id#5':'5'})

In [11]:
# append all sample.submitters (sample_ID col) 
map_df = map_df.drop(columns = ['type','project_id','analyte_type']) # only for protein, need to get files for phospho
melted = map_df.melt(id_vars = 'aliquot_ID', value_name = 'sample_ID')
all_vals = melted.loc[melted.sample_ID != ' '] # drop empty vals for samples
map_df_1 = all_vals[['aliquot_ID','sample_ID']]
map_df_1

<ipython-input-11-8b173d374f8c>:3: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  melted = map_df.melt(id_vars = 'aliquot_ID', value_name = 'sample_ID')


,aliquot_ID,sample_ID
0,CPT0053040004,9f905736-f662-41d6-b3ac-16758d
1,CPT0052940004,93e30fd5-e57e-4503-a175-863c7d
2,CPT0052170004,5a84eae1-197e-4463-ad65-59becc
3,CPT0051690004,2f2e5477-42a4-4906-a943-bf7f80
4,CPT0001580009,C3L-00001-02
...,...,...
10498,CPT0265630004,C3L-04090-04
11352,CPT0117010004,C3N-01871-04
13296,CPT0192540004,C3L-02665-05
13429,CPT0190360004,C3L-03407-05


In [12]:
map_df_1.sample_ID.duplicated().value_counts() #unique values for sample_IDs equals the number of sample_IDs in the pdc_sample file

False    2870
True      548
Name: sample_ID, dtype: int64

In [17]:
map_df_1.aliquot_ID.duplicated().value_counts() #some duplicate sample_IDs have different aliquots

False    3075
True      343
Name: aliquot_ID, dtype: int64

Get df using sample IDs as the index to merge with.

In [13]:
mapper_2 = 'pdc_sample_2021-03-05_16_43.tsv.txt'
m_path_2 = "../../../input/"+mapper_2

In [15]:
map_df_2 = pd.read_csv(m_path_2, sep = "\t", na_values = 'NA') # type, project_id, and analyte_type have all same vals

# Make patient_ID vals (case_id + .N if normal sample) 
map_df_2['patient_ID'] = map_df_2['cases.submitter_id'] +'_'+ map_df_2['tissue_type']
map_df_2['patient_ID'] = map_df_2['patient_ID'].str.replace('_Tumor$','', regex=True)
map_df_2['patient_ID'] = map_df_2['patient_ID'].str.replace('_Normal$','.N', regex=True)
map_df_2 = map_df_2.rename(columns = {'submitter_id': 'sample_ID'})
map_df_2 = map_df_2[['sample_ID', 'patient_ID']]
map_df_2

,sample_ID,patient_ID
0,9f905736-f662-41d6-b3ac-16758d,11LU013
1,93e30fd5-e57e-4503-a175-863c7d,11LU016
2,5a84eae1-197e-4463-ad65-59becc,11LU022
3,2f2e5477-42a4-4906-a943-bf7f80,11LU035
4,C3L-00001-02,C3L-00001
...,...,...
2865,C3N-05923-01,C3N-05923
2866,C3N-05923-09,C3N-05923.N
2867,C3N-05929-03,C3N-05929
2868,C3N-05929-05,C3N-05929.N


In [16]:
# Check if any sample_IDs are only in one file
test = 'sample_ID'
s1 = list(map_df_1[test])
s2 = list(map_df_2[test])

n = set(s1) ^ set(s2) 
print('unique vals:', len(n), '\n')

in_s1 = []
in_s2 = []

for e in n: 
    if e in s1:
        #print('in s1')
        in_s1.append(e)
    if e in s2:
        #print('in s2')
        in_s2.append(e)
print('num in s1:', len(in_s1))
print('num in s2:', len(in_s2))

unique vals: 0 

num in s1: 0
num in s2: 0


In [17]:
print('total length of aliquot to sample file:', len(map_df_1))
print('total length of sample to patient file:', len(map_df_2))
print('difference:', len(map_df_1) - len(map_df_2))

total length of aliquot to sample file: 3418
total length of sample to patient file: 2870
difference: 548


In [18]:
# Merge dfs
# set index to sample_id
map_df_1 = map_df_1.set_index('sample_ID')
map_df_2 = map_df_2.set_index('sample_ID')
all_df = map_df_1.join(map_df_2, how = 'outer') 

In [19]:
# drop duplicate aliquot_IDs (keeps first aliquot of duplicated)
all_df = all_df.loc[~ all_df.aliquot_ID.duplicated()]
print('length:', len(all_df))

length: 3075


Add GBM normal samples to matched_ids dictionary.

In [20]:
# GBM normal samples
gbm_file = 'GBM_normal_sample_mapping.xlsx'
g_path = "../../../input/"+gbm_file

In [22]:
gbm_df = pd.read_excel(g_path, na_values = 'NA') 
gbm_df = gbm_df[['BSIID', 'Subject ID']].rename(columns = {'BSIID':'aliquot_ID', 'Subject ID':'patient_ID'})
all_df = all_df.append(gbm_df)
print(len(gbm_df))

17


In [23]:
print('total length of combined mapping file:', len(all_df)) # 3075 + 17 (gbm normal) = 3092

total length of combined mapping file: 3092


# Create file

In [24]:
all_df.to_csv('../../../input/aliquot_to_patient_ID.tsv', sep = '\t')

In [25]:
mapping_df = pd.read_csv('../../../input/aliquot_to_patient_ID.tsv', delimiter = '\t', index_col = 0)

In [26]:
mapping_df[mapping_df.patient_ID.str.contains('C3L-03407')]

,aliquot_ID,patient_ID
C3L-03407-01,CPT0190360004,C3L-03407
C3L-03407-01,CPT0190360012,C3L-03407


In [27]:
# Check if key values will be unique
if True in set(mapping_df.aliquot_ID.duplicated()):
    print('There are duplicates. Key vals will not be unique.')

In [28]:
a=mapping_df['aliquot_ID']
print('total rows',len(a))
len(a.dropna())

total rows 3092


3092